# Hearth Stone Decker

* https://hsreplay.net/decks/

In [4]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import pandas as pd
import numpy as np
import time
import re
import datetime
import sqlite3 as lite
import os
import random
import calendar

In [5]:
url = "https://hsreplay.net/decks/"

In [6]:
# Open Browser and open udn library #打開瀏覽器，但不要載入圖片
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values': {
        #'images': 2,
        #'javascript': 2
    }
}
options.add_experimental_option('prefs', prefs)
options.add_experimental_option('prefs', prefs)

# 偽裝header
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15"
options.add_argument('--user-agent=%s' % user_agent)
# 防止 javascript detect selenium
options.add_experimental_option('excludeSwitches', ['enable-automation'])
# headless
#options.add_argument('-headless')

global driver

In [7]:
driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', chrome_options=options)

<ipython-input-7-240008e0e6e7>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', chrome_options=options)
<ipython-input-7-240008e0e6e7>:1: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver', chrome_options=options)


In [8]:
driver.get(url)

# Deck Dictionary

In [9]:
occupation_dict = {
    "DemonHunter" : "https://hsreplay.net/decks/#playerClasses=DEMONHUNTER",
    "Druid" : "https://hsreplay.net/decks/#playerClasses=DRUID",
    "Hunter" : "https://hsreplay.net/decks/#playerClasses=HUNTER",
    "Mage" : "https://hsreplay.net/decks/#playerClasses=MAGE",
    "Paladin" : "https://hsreplay.net/decks/#playerClasses=PALADIN",
    "Priest" : "https://hsreplay.net/decks/#playerClasses=PRIEST",
    "Rogue" : "https://hsreplay.net/decks/#playerClasses=ROGUE",
    "Shaman" : "https://hsreplay.net/decks/#playerClasses=SHAMAN",
    "Warlock" : "https://hsreplay.net/decks/#playerClasses=WARLOCK",
    "Warrior" : "https://hsreplay.net/decks/#playerClasses=WARRIOR",
}

In [10]:
driver.get(occupation_dict["DemonHunter"])

In [11]:
# 滑到網頁最下方
def scroll_down():
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

### 拿牌組的key 

因為名稱不唯一，而一個牌組是由其內的牌來組成的，所以拿key比拿名稱重要。
名稱就只是這個牌組的一個feature而已

In [10]:
# Load
scroll_down()

DemonHunter_deck_dict = {}

# get keys (every 18 in 1 page, for 12 pages)
for deck_key in driver.find_elements_by_partial_link_text("Demon Hunter"):
    #print(deck_key.get_attribute("href"))
    href = deck_key.get_attribute("href")
    key = href.split("/")[-2]
    DemonHunter_deck_dict[key] = {"URL" : href}

NameError: name 'scroll_down' is not defined

In [10]:
DemonHunter_deck_dict

{'kyCUK1H2Y5wtS52RdFQLXb': {'URL': 'https://hsreplay.net/decks/kyCUK1H2Y5wtS52RdFQLXb/'},
 'wKwIEzbviVLvEj6pEP4Coe': {'URL': 'https://hsreplay.net/decks/wKwIEzbviVLvEj6pEP4Coe/'},
 'DqUe87qVw99Dso2poGdBif': {'URL': 'https://hsreplay.net/decks/DqUe87qVw99Dso2poGdBif/'},
 'QWLOmOj7uYAGtA5hzIVrTe': {'URL': 'https://hsreplay.net/decks/QWLOmOj7uYAGtA5hzIVrTe/'},
 '4aChHKfoy145otte4P8r1f': {'URL': 'https://hsreplay.net/decks/4aChHKfoy145otte4P8r1f/'},
 'nsEeXHKmqGKVT1yjrULMUe': {'URL': 'https://hsreplay.net/decks/nsEeXHKmqGKVT1yjrULMUe/'},
 'E2oVtowA2KLswjWDSBERog': {'URL': 'https://hsreplay.net/decks/E2oVtowA2KLswjWDSBERog/'},
 'wRHNhCjTRm0eEjtMrsWGod': {'URL': 'https://hsreplay.net/decks/wRHNhCjTRm0eEjtMrsWGod/'},
 'zx3oqTdpYeRHgRE5Xpp7Dh': {'URL': 'https://hsreplay.net/decks/zx3oqTdpYeRHgRE5Xpp7Dh/'},
 'YwB5GcKa0OMN33KGxBZpEb': {'URL': 'https://hsreplay.net/decks/YwB5GcKa0OMN33KGxBZpEb/'},
 '6rxb2ldIwEkkCVfOePXaJe': {'URL': 'https://hsreplay.net/decks/6rxb2ldIwEkkCVfOePXaJe/'},
 'ZRJhHAU9

### 拿到牌組的其他feature

1. Name

2. Composition (1 or 2 cards)

3. Card URL

4. Overall Win Rate

5. 合成成本 (表示土豪程度)

6. 已遊玩的局數 (表示熱門程度)

7. 遊戲持續時間 (表示節奏)

先棄用
```python
# row 1~18 是牌組

# Name
soup.select(".row")[1].h3.text

### Composition

# Card Name
soup.select(".row")[2].text
# Num of Cards
soup.select(".row")[2].text
```

```python
# get deck names
deck_names = []
for deck_name in driver.find_elements_by_css_selector(".deck-name"):
    #print(deck_name.text)
    deck_names.append(deck_name.text)
```

### 寫成對於第一頁中的第i個牌組（一頁有18個牌組）

```python
soup = BeautifulSoup(driver.page_source, 'lxml')
for i in range(0,18):
    keyname = list(DemonHunter_deck_dict.keys())[i]

    # get deck names
    deck_name = driver.find_elements_by_css_selector(".deck-name")[i]
        #print(deck_name.text)
    DemonHunter_deck_dict[keyname]["Deck_Name"] = deck_name.text

    # Get Card Compositions
    DemonHunter_deck_dict[keyname]["Deck_Composition"] = {}
    card_list = driver.find_elements_by_css_selector("ul.card-list")[i].find_elements_by_css_selector("a")
    for j in range(len(card_list)): #從第一張卡到最後一張卡
        #print(card_url.get_attribute("href"))
        card_url = card_list[j].get_attribute("href")
        DemonHunter_deck_dict[keyname]["Deck_Composition"][card_url] = soup.select("ul.card-list")[i].select(".card-icon")[j]["aria-label"]
        

#     # 張數 #注意這是只有第一個牌組的組成，還沒有抓剩下17個
#     for card_icon in driver.find_element_by_css_selector("ul.card-list").find_elements_by_css_selector(".card-icon"):
#         print(card_icon.text)
```

```python
DemonHunter_deck_dict["kyCUK1H2Y5wtS52RdFQLXb"]
{'URL': 'https://hsreplay.net/decks/kyCUK1H2Y5wtS52RdFQLXb/',
 'Deck_Name': 'Aggro Demon Hunter',
 'Deck_Composition': {'https://hsreplay.net/cards/57182/': '守護者強化商人 ×2',
  'https://hsreplay.net/cards/58972/': '強悍的新生 ×2',
  'https://hsreplay.net/cards/59394/': '惡魔夥伴 ×2',
  'https://hsreplay.net/cards/58487/': '戰鬥犬魔 ×2',
  'https://hsreplay.net/cards/58610/': '法力燃燒 ×2',
  'https://hsreplay.net/cards/55161/': '烈焰戰鬥法師 ×2',
  'https://hsreplay.net/cards/54269/': '開朗小幫手 ×2',
  'https://hsreplay.net/cards/56928/': '雙刃斬 ×2',
  'https://hsreplay.net/cards/57175/': '噬骨打鬥者 ×2',
  'https://hsreplay.net/cards/56587/': '幽暗之翼 ×2',
  'https://hsreplay.net/cards/61898/': '蠕動懼怪 ×2',
  'https://hsreplay.net/cards/59590/': '閱讀狂 ×2',
  'https://hsreplay.net/cards/61273/': '空中特技 ×2',
  'https://hsreplay.net/cards/61133/': '驚懼領主之噬 ×2',
  'https://hsreplay.net/cards/61135/': '高蹺手 ×2'}}
```

### 更好的存法

```python
"Deck_Composition" : {
    58972 : {
        "duo" : True, #表達是否放兩張，用1存是
        "card_name" : "守護者強化商人"
    }
}

```

```python
soup = BeautifulSoup(driver.page_source, 'lxml')
for i in range(0,18):
    keyname = list(DemonHunter_deck_dict.keys())[i]

    # get deck names
    deck_name = driver.find_elements_by_css_selector(".deck-name")[i]
    DemonHunter_deck_dict[keyname]["Deck_Name"] = deck_name.text

    # Get Card Compositions
    DemonHunter_deck_dict[keyname]["Deck_Composition"] = {}
    card_list = driver.find_elements_by_css_selector("ul.card-list")[i].find_elements_by_css_selector("a")
    #Key for Cards
    card_urls = [card_url.get_attribute("href") for card_url in card_list]
    card_keys = [int(k.split("/")[-2]) for k in card_urls]
    
    for j in range(len(card_keys)): #從第一張卡到最後一張卡
        name_num_str = soup.select("ul.card-list")[i].select(".card-icon")[j]["aria-label"]
        card_name = name_num_str.split()[0]
        if name_num_str.split()[-1] == "★":
            IsDuo = 0
        if name_num_str.split()[-1] == "×2":
            IsDuo = 1
        
        DemonHunter_deck_dict[keyname]["Deck_Composition"][card_keys[j]] = {
            "card_name" : card_name,
            "IsDuo" : IsDuo,
            "card_URL" : card_urls[j],
        }
        
DemonHunter_deck_dict["kyCUK1H2Y5wtS52RdFQLXb"]
        
{'URL': 'https://hsreplay.net/decks/kyCUK1H2Y5wtS52RdFQLXb/',
 'Deck_Name': 'Aggro Demon Hunter',
 'Deck_Composition': {57182: {'card_name': '守護者強化商人',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/57182/'},
  58972: {'card_name': '強悍的新生',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/58972/'},
  59394: {'card_name': '惡魔夥伴',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/59394/'},
  58487: {'card_name': '戰鬥犬魔',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/58487/'},
  58610: {'card_name': '法力燃燒',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/58610/'},
  55161: {'card_name': '烈焰戰鬥法師',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/55161/'},
  54269: {'card_name': '開朗小幫手',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/54269/'},
  56928: {'card_name': '雙刃斬',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/56928/'},
  57175: {'card_name': '噬骨打鬥者',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/57175/'},
  56587: {'card_name': '幽暗之翼',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/56587/'},
  61898: {'card_name': '蠕動懼怪',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/61898/'},
  59590: {'card_name': '閱讀狂',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/59590/'},
  61273: {'card_name': '空中特技',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/61273/'},
  61133: {'card_name': '驚懼領主之噬',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/61133/'},
  61135: {'card_name': '高蹺手',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/61135/'}}}
```

## 重寫dict增加資料的方式 加feature

Overall Win Rate

合成成本 (表示土豪程度)

已遊玩的局數 (表示熱門程度)

遊戲持續時間 (表示節奏)

In [11]:
soup = BeautifulSoup(driver.page_source, 'lxml')
for i in range(0,18):
    keyname = list(DemonHunter_deck_dict.keys())[i]

    # get deck names
    deck_name = driver.find_elements_by_css_selector(".deck-name")[i]
    DemonHunter_deck_dict[keyname]["Deck_Name"] = deck_name.text

    # Get Card Compositions
    DemonHunter_deck_dict[keyname]["Deck_Composition"] = {}
    card_list = driver.find_elements_by_css_selector("ul.card-list")[i].find_elements_by_css_selector("a")
    #Key for Cards
    card_urls = [card_url.get_attribute("href") for card_url in card_list]
    card_keys = [int(k.split("/")[-2]) for k in card_urls]
    
    for j in range(len(card_keys)): #從第一張卡到最後一張卡
        name_num_str = soup.select("ul.card-list")[i].select(".card-icon")[j]["aria-label"]
        card_name = name_num_str.split()[0]
        if name_num_str.split()[-1] == "★":
            IsDuo = 0
        if name_num_str.split()[-1] == "×2":
            IsDuo = 1
        
        DemonHunter_deck_dict[keyname]["Deck_Composition"][card_keys[j]] = {
            "card_name" : card_name,
            "IsDuo" : IsDuo,
            "card_URL" : card_urls[j],
        }

    # Win Rate
    win_rate = driver.find_elements_by_css_selector(".win-rate")[i].text
    DemonHunter_deck_dict[keyname]["win_rate"] = win_rate
    
    # 合卡成本
    dust_cost = driver.find_elements_by_css_selector(".dust-cost")[i].text
    DemonHunter_deck_dict[keyname]["dust_cost"] = dust_cost
    
    # 該牌組已被使用的對場場數
    game_count = driver.find_elements_by_css_selector(".game-count")[i].text
    DemonHunter_deck_dict[keyname]["game_count"] = game_count
    
    # 每回比賽平均時間
    time_duration = driver.find_elements_by_css_selector(".duration")[i].text
    DemonHunter_deck_dict[keyname]["time_duration"] = time_duration
    

In [13]:
DemonHunter_deck_dict["kyCUK1H2Y5wtS52RdFQLXb"]

{'URL': 'https://hsreplay.net/decks/kyCUK1H2Y5wtS52RdFQLXb/',
 'Deck_Name': 'Aggro Demon Hunter',
 'Deck_Composition': {57182: {'card_name': '守護者強化商人',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/57182/'},
  58972: {'card_name': '強悍的新生',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/58972/'},
  59394: {'card_name': '惡魔夥伴',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/59394/'},
  58487: {'card_name': '戰鬥犬魔',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/58487/'},
  58610: {'card_name': '法力燃燒',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/58610/'},
  55161: {'card_name': '烈焰戰鬥法師',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/55161/'},
  54269: {'card_name': '開朗小幫手',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/54269/'},
  56928: {'card_name': '雙刃斬',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/cards/56928/'},
  57175: {'card_name': '噬骨打鬥者',
   'IsDuo': 1,
   'card_URL': 'https://hsreplay.net/

# function

寫成迴圈，對於12頁的資料都如此爬

In [12]:
occupation_dict["DemonHunter"]

'https://hsreplay.net/decks/#playerClasses=DEMONHUNTER'

In [13]:
# Raise Error
def getDeck(occupation_name = "DemonHunter"):
    
    DemonHunter_deck_dict = {}
    
    for k in range(1,12+1):
        page_url = f"{occupation_dict[occupation_name]}%page={str(k)}"

        driver.get(page_url)
        driver.implicitly_wait(20)
        # Load
        scroll_down()

        #DemonHunter_deck_dict = {}

        # get keys (every 18 in 1 page, for 12 pages)
        for deck_key in driver.find_elements_by_partial_link_text("Demon Hunter"):
            #print(deck_key.get_attribute("href"))
            href = deck_key.get_attribute("href")
            key = href.split("/")[-2]
            #DemonHunter_deck_dict[key] = {"URL" : href}
            DemonHunter_deck_dict[key]["URL"] = href

        soup = BeautifulSoup(driver.page_source, 'lxml')
        for i in range(0,18):
            keyname = list(DemonHunter_deck_dict.keys())[i]

            # get deck names
            deck_name = driver.find_elements_by_css_selector(".deck-name")[i]
            DemonHunter_deck_dict[keyname]["Deck_Name"] = deck_name.text

            # Get Card Compositions
            DemonHunter_deck_dict[keyname]["Deck_Composition"] = {}
            card_list = driver.find_elements_by_css_selector("ul.card-list")[i].find_elements_by_css_selector("a")
            #Key for Cards
            card_urls = [card_url.get_attribute("href") for card_url in card_list]
            card_keys = [int(k.split("/")[-2]) for k in card_urls]

            for j in range(len(card_keys)): #從第一張卡到最後一張卡
                name_num_str = soup.select("ul.card-list")[i].select(".card-icon")[j]["aria-label"]
                card_name = name_num_str.split()[0]
                if name_num_str.split()[-1] == "★":
                    IsDuo = 0
                if name_num_str.split()[-1] == "×2":
                    IsDuo = 1

                DemonHunter_deck_dict[keyname]["Deck_Composition"][card_keys[j]] = {
                    "card_name" : card_name,
                    "IsDuo" : IsDuo,
                    "card_URL" : card_urls[j],
                }

            # Win Rate
            win_rate = driver.find_elements_by_css_selector(".win-rate")[i].text
            DemonHunter_deck_dict[keyname]["win_rate"] = win_rate

            # 合卡成本
            dust_cost = driver.find_elements_by_css_selector(".dust-cost")[i].text
            DemonHunter_deck_dict[keyname]["dust_cost"] = dust_cost

            # 該牌組已被使用的對場場數
            game_count = driver.find_elements_by_css_selector(".game-count")[i].text
            DemonHunter_deck_dict[keyname]["game_count"] = game_count

            # 每回比賽平均時間
            time_duration = driver.find_elements_by_css_selector(".duration")[i].text
            DemonHunter_deck_dict[keyname]["time_duration"] = time_duration
            
            time.sleep(1)

    return DemonHunter_deck_dict

In [ ]:
def getDeck(occupation_name = "DemonHunter"):
    
    # Overall_dict
    deck_dict = {}
    
    for k in range(1,12+1):
        page_url = f"{occupation_dict[occupation_name]}%page={str(k)}"

        driver.get(page_url)
        driver.implicitly_wait(20)
        # Load
        scroll_down()
        
        # a sub dict
        DemonHunter_deck_dict = {}

        # get keys (every 18 in 1 page, for 12 pages)
        for deck_key in driver.find_elements_by_partial_link_text("Demon Hunter"):
            #print(deck_key.get_attribute("href"))
            href = deck_key.get_attribute("href")
            key = href.split("/")[-2]
            DemonHunter_deck_dict[key] = {"URL" : href}

        soup = BeautifulSoup(driver.page_source, 'lxml')
        for i in range(0,18):
            keyname = list(DemonHunter_deck_dict.keys())[i]

            # get deck names
            deck_name = driver.find_elements_by_css_selector(".deck-name")[i]
            DemonHunter_deck_dict[keyname]["Deck_Name"] = deck_name.text

            # Get Card Compositions
            DemonHunter_deck_dict[keyname]["Deck_Composition"] = {}
            card_list = driver.find_elements_by_css_selector("ul.card-list")[i].find_elements_by_css_selector("a")
            #Key for Cards
            card_urls = [card_url.get_attribute("href") for card_url in card_list]
            card_keys = [int(k.split("/")[-2]) for k in card_urls]

            for j in range(len(card_keys)): #從第一張卡到最後一張卡
                name_num_str = soup.select("ul.card-list")[i].select(".card-icon")[j]["aria-label"]
                card_name = name_num_str.split()[0]
                if name_num_str.split()[-1] == "★":
                    IsDuo = 0
                if name_num_str.split()[-1] == "×2":
                    IsDuo = 1

                DemonHunter_deck_dict[keyname]["Deck_Composition"][card_keys[j]] = {
                    "card_name" : card_name,
                    "IsDuo" : IsDuo,
                    "card_URL" : card_urls[j],
                }

            # Win Rate
            win_rate = driver.find_elements_by_css_selector(".win-rate")[i].text
            DemonHunter_deck_dict[keyname]["win_rate"] = win_rate

            # 合卡成本
            dust_cost = driver.find_elements_by_css_selector(".dust-cost")[i].text
            DemonHunter_deck_dict[keyname]["dust_cost"] = dust_cost

            # 該牌組已被使用的對場場數
            game_count = driver.find_elements_by_css_selector(".game-count")[i].text
            DemonHunter_deck_dict[keyname]["game_count"] = game_count

            # 每回比賽平均時間
            time_duration = driver.find_elements_by_css_selector(".duration")[i].text
            DemonHunter_deck_dict[keyname]["time_duration"] = time_duration
            
            time.sleep(1)
            # Merge to the parent dict
            deck_dict.update(DemonHunter_decks)

    return DemonHunter_deck_dict

In [14]:
DemonHunter_decks = getDeck(occupation_name = "DemonHunter")

In [17]:
len(DemonHunter_decks.keys())

18

In [18]:
deck_dict = {}

In [20]:
deck_dict.update(DemonHunter_decks)

## 進入一個牌組內爬資訊

In [112]:
for key in list(DemonHunter_deck_dict.keys())[0:1]:
    driver.get(DemonHunter_deck_dict[key]['URL'])
    driver.implicitly_wait(10)

In [113]:
sub_soup = BeautifulSoup(driver.page_source, 'lxml')

In [117]:
sub_soup.select(".card-name")

[<figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">守護者強化商人</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">強悍的新生</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">惡魔夥伴</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">戰鬥犬魔</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">法力燃燒</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">烈焰戰鬥法師</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">開朗小幫手</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">雙刃斬</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 28px);">噬骨打鬥者</figcaption>,
 <figcaption class="card-name" style="font-size: 0.9em; width: calc(100% - 

In [ ]:
for card in driver.find_elements_by_css_selector(".card-name"):
    print(card.attrs)

In [18]:
j = 0
deck_list = driver.find_elements_by_css_selector("ul.card-list")
for deck in deck_list:
    j+=1
    print(f"Deck {j}")
    for card_url in deck.find_elements_by_css_selector("a"):
        print(card_url.get_attribute("href"))

Deck 1
https://hsreplay.net/cards/57182/
https://hsreplay.net/cards/58972/
https://hsreplay.net/cards/59394/
https://hsreplay.net/cards/58487/
https://hsreplay.net/cards/58610/
https://hsreplay.net/cards/55161/
https://hsreplay.net/cards/54269/
https://hsreplay.net/cards/56928/
https://hsreplay.net/cards/57175/
https://hsreplay.net/cards/56587/
https://hsreplay.net/cards/61898/
https://hsreplay.net/cards/59590/
https://hsreplay.net/cards/61273/
https://hsreplay.net/cards/61133/
https://hsreplay.net/cards/61135/
Deck 2
https://hsreplay.net/cards/58452/
https://hsreplay.net/cards/56928/
https://hsreplay.net/cards/59724/
https://hsreplay.net/cards/58492/
https://hsreplay.net/cards/56791/
https://hsreplay.net/cards/59725/
https://hsreplay.net/cards/59259/
https://hsreplay.net/cards/56583/
https://hsreplay.net/cards/61125/
https://hsreplay.net/cards/60151/
https://hsreplay.net/cards/61136/
https://hsreplay.net/cards/58202/
https://hsreplay.net/cards/59089/
https://hsreplay.net/cards/59732/


# Define Function

In [19]:
#def getDeck(deck_class = "DemonHunter", ):
for k in range(1, 7):
    url = deck_dict[deck_class]+f"&page={k}"
    print(url)

NameError: name 'deck_class' is not defined